<a href="https://colab.research.google.com/github/paolobalasso/YoutubeTrading/blob/main/HedgedvsUnheadged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install yfinance
!pip install prophet

%matplotlib inline

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
from prophet import Prophet



In [ ]:
#EXCHANGE RATE

exchange_df = yf.download(['EURUSD=X','GBPUSD=X','EURGBP=X'],'2004-01-04',end = '2026-01-01')['Close']
(exchange_df.pct_change()+1).cumprod().plot()

In [ ]:
#MSCI WORLD

%matplotlib inline
prices = yf.download(['IWDE.AS','IWDA.AS'],"2022-10-04",end = '2026-01-01')['Adj Close']
(prices['IWDA.AS'].multiply(yf.download(['EURUSD=X'],'2022-10-04',end = '2024-01-01')['Close'],axis="index").pct_change()+1).cumprod().plot(label ='Perfect Hedged',figsize=(10,5))
(prices['IWDE.AS'].pct_change()+1).cumprod().plot(label ='Hedged-IWDE.AS')
(prices['IWDA.AS'].pct_change()+1).cumprod().plot(label ='Unhedged-IWDA.AS')
plt.legend( loc ="lower right")
plt.show()
(yf.download(['EURUSD=X'],'2022-10-04',end = '2024-01-01')['Close'].pct_change()+1).cumprod().plot(figsize=(10,5))


In [ ]:
%matplotlib inline
prices = yf.download(['IWDE.AS','IWDA.AS'],"2022-10-04",end = '2026-01-01')['Adj Close']
prices['IWDA.PerfectHedged'] = prices['IWDA.AS'].multiply(yf.download(['EURUSD=X'],'2022-10-04',end = '2026-01-01')['Close'],axis="index")

(prices.pct_change()+1).cumprod()

In [ ]:
#BOND ETFS
%matplotlib inline
prices = yf.download(['DTLE.L','IDTL.L'],"2022-10-04",end = '2026-01-01')['Adj Close']
(prices['IDTL.L'].multiply(yf.download(['EURUSD=X'],'2022-10-04',end = '2026-01-01')['Close'],axis="index").pct_change()+1).cumprod().plot(label ='Perfect Hedged',figsize=(10,5))
(prices['DTLE.L'].pct_change()+1).cumprod().plot(label ='DTLE.L-Headged')
(prices['IDTL.L'].pct_change()+1).cumprod().plot(label ='IDTL.L-Unheadged')
plt.legend( loc ="lower right")
plt.show()
(yf.download(['EURUSD=X'],'2022-10-04',end = '2026-01-01')['Close'].pct_change()+1).cumprod().plot(figsize=(10,5))
plt.show()

prices['IDTL.PerfectHedged'] = prices['IDTL.L'].multiply(yf.download(['EURUSD=X'],'2022-10-04',end = '2026-01-01')['Close'],axis="index")
(prices.pct_change()+1).cumprod()**(1)


In [ ]:
#BOND ETFS
%matplotlib inline
prices = yf.download(['DTLE.L','IS04.DE'],"2019-10-04",end = '2024-01-01')['Adj Close']
(prices['IS04.DE'].multiply(yf.download(['EURUSD=X'],'2019-10-04',end = '2024-01-01')['Close'],axis="index").pct_change()+1).cumprod().plot(label ='Perfect Hedged',figsize=(10,5))
(prices['DTLE.L'].pct_change()+1).cumprod().plot(label ='DTLE.L-Headged')
(prices['IS04.DE'].pct_change()+1).cumprod().plot(label ='IDTL.L-Unheadged')
plt.legend( loc ="lower right")
plt.show()
(yf.download(['EURUSD=X'],'2019-10-04',end = '2024-01-01')['Close'].pct_change()+1).cumprod().plot(figsize=(10,5))
plt.show()

prices['IDTL.PerfectHedged'] = prices['IS04.DE'].multiply(yf.download(['EURUSD=X'],'2019-10-04',end = '2024-01-01')['Close'],axis="index")
(prices.pct_change()+1).cumprod()**(1/4)


In [ ]:
#MSCI US
%matplotlib inline
prices = yf.download(['IUSA.AS','IUES.AS'],"2022-10-04",end = '2024-01-01')['Adj Close']
(prices['IUSA.AS'].multiply(yf.download(['EURUSD=X'],'2022-10-04',end = '2024-01-01')['Close'],axis="index").pct_change()+1).cumprod().plot(label ='Perfect Hedged',figsize=(10,5))
(prices['IUES.AS'].pct_change()+1).cumprod().plot(label ='IUSE.AS-Headged',figsize=(10,5))
(prices['IUSA.AS'].pct_change()+1).cumprod().plot(label ='IUSA.AS-Unheadged',figsize=(10,5))
plt.legend( loc ="lower right")
plt.show()
(yf.download(['EURUSD=X'],'2022-10-04',end = '2024-01-01')['Close'].pct_change()+1).cumprod().plot()



In [ ]:
%matplotlib inline
prices = yf.download(['IUSA.AS','IUES.AS'],"2022-10-04",end = '2024-01-01')['Adj Close']
prices['IUSA.PerfectHedged'] = prices['IUSA.AS'].multiply(yf.download(['EURUSD=X'],'2019-10-04',end = '2024-01-01')['Close'],axis="index")

(prices.pct_change()+1).cumprod()

In [ ]:
#### Prediction

"""
Functions to evaluate the model
"""
def MAPE(y, y_hat):
        return np.mean(np.abs((y - y_hat)/y)) * 100

def RMSE(y, y_hat):
    return np.sqrt(np.mean(np.square(y - y_hat)))

"""
Function to perform a univariate forecast and evaluate performance
"""
def uni_forecast(slug_df_lst, slug='EURUSD=X', plot=False, plot_comp=False):
    print('='*40)
    new = exchange_df_w['EURUSD=X'].reset_index()[['Date', 'EURUSD=X']]
    new = new.rename(columns={'Date': 'ds', 'EURUSD=X': 'y'})

    test_data = new.loc[(len(new)-52):len(new)]
    train_data = new.drop(test_data.index)
    print(f'Training data shape: {train_data.shape}')
    print(f'Test data shape: {test_data.shape}')

    model = Prophet(seasonality_mode='multiplicative')
    model.fit(train_data)

    future = test_data[['ds']]
    forecast = model.predict(future)
    print(f'Forecast data shape: {forecast.shape}')

    if plot == True:
          model.plot(forecast)
          exchange_df_w['EURUSD=X'].iloc[(len(new)-52):len(new)].plot(label='Test',color='black')
          plt.show()
    if plot_comp == True:
          model.plot_components(forecast)
          plt.show()

    mape = str(round(MAPE(test_data['y'].values, forecast['yhat'].values),2)) + "%"
    rmse = round(RMSE(test_data['y'].values, forecast['yhat'].values), 5)
    print(f'MAPE: {mape}')
    print(f'RMSE: {rmse}')


In [ ]:
exchange_df_w = exchange_df.resample("W-FRI").last()
uni_forecast(exchange_df_w, slug='EURUSD=X', plot=True, plot_comp=False)

In [ ]:
exchange_df_w = exchange_df.resample("W-FRI").last()
new = exchange_df_w['EURUSD=X'].reset_index()[['Date', 'EURUSD=X']]
new = new.rename(columns={'Date': 'ds', 'EURUSD=X': 'y'})
m = Prophet(seasonality_mode='multiplicative',yearly_seasonality=True).fit(new)
future = m.make_future_dataframe(periods=104,freq="W-FRI")
fcst = m.predict(future)
fig = m.plot(fcst)
